# Scaling Law

Task: Estimate the performance (box AP) at future steps, given a history of (box AP, number of steps).

## Setup

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import scipy.stats as stats

## Code

In [2]:
data = pd.read_csv('assets/wandb_export.csv').drop(["Name", "_wandb"], axis=1)
# Coumns: [Step, bbox/AP]
data = data.sort_values(by="Step")

In [3]:
# Define custom functions
def log_func(x, a, b):
    return a * np.log(x) + b

def log_linear_func(x, a, b, c):
    return a * np.log(x) + b * x + c

def power_func(x, a, b, c):
    return a - b * x**(-c)

In [31]:
colors = {
    "red": '#%02x%02x%02x' % (255, 59, 48),
    "orange": '#%02x%02x%02x' % (255, 149, 0),
    "yellow": '#%02x%02x%02x' % (255, 204, 0),
    "green": '#%02x%02x%02x' % (40, 205, 65),
    "mint": '#%02x%02x%02x' % (0, 199, 190),
    "teal": '#%02x%02x%02x' % (89, 173, 196),
    "cyan": '#%02x%02x%02x' % (85, 190, 240),
    "blue": '#%02x%02x%02x' % (0, 122, 255),
    "indigo": '#%02x%02x%02x' % (88, 86, 214),
    "purple": '#%02x%02x%02x' % (175, 82, 222),
    "pink": '#%02x%02x%02x' % (255, 45, 85),
    "brown": '#%02x%02x%02x' % (162, 132, 94),
    "gray": '#%02x%02x%02x' % (142, 142, 147),
    "black": '#%02x%02x%02x' % (0, 0, 0),
}

In [37]:
# Extract step and bbox/AP columns
steps = data['Step'].values
ap_scores = data['bbox/AP'].values

# Fit the functions
popt_log, _ = curve_fit(log_func, steps, ap_scores)
popt_log_linear, _ = curve_fit(log_linear_func, steps, ap_scores)
popt_power, _ = curve_fit(power_func, steps, ap_scores, p0=[50, 50, 0.1])

# Generate points for the fitted curves
x_fit = np.linspace(steps.min(), 120000, 1000)
y_fit_log = log_func(x_fit, *popt_log)
y_fit_log_linear = log_linear_func(x_fit, *popt_log_linear)
y_fit_power = power_func(x_fit, *popt_power)

# Create forecasts
future_steps = np.array([90000, 100000, 110000, 120000])
log_forecast = log_func(future_steps, *popt_log)
log_linear_forecast = log_linear_func(future_steps, *popt_log_linear)
power_forecast = power_func(future_steps, *popt_power)

fig = px.scatter(data, x="Step", y="bbox/AP")

# Update layout
fig.update_layout(
    title='Object Detection Model Performance with different model fits',
    xaxis_title='Steps',
    yaxis_title='bbox/AP',
    legend_title='Legend',
    hovermode="x"
)
fig.add_scatter(x=x_fit, y=y_fit_log, mode='lines', name='Logarithmic fit')
fig.add_scatter(x=x_fit, y=y_fit_log_linear, mode='lines', name='Log-Linear fit')
fig.add_scatter(x=x_fit, y=y_fit_power, mode='lines', name='Power law fit')
fig.show()

# Print forecasts
print("Logarithmic Forecast:")
for step, forecast in zip(future_steps, log_forecast):
    print(f"Step {step}: {forecast:.4f}")

print("\nLog-Linear Forecast:")
for step, forecast in zip(future_steps, log_linear_forecast):
    print(f"Step {step}: {forecast:.4f}")

print("\nPower law Forecast:")
for step, forecast in zip(future_steps, power_forecast):
    print(f"Step {step}: {forecast:.4f}")

plt.show()

# Calculate Mean Squared Error for each model
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

mse_log = mse(ap_scores, log_func(steps, *popt_log))
mse_log_linear = mse(ap_scores, log_linear_func(steps, *popt_log_linear))
mse_power = mse(ap_scores, power_func(steps, *popt_power))

print("\nMean Squared Error:")
print(f"Logarithmic: {mse_log:.6f}")
print(f"Power law: {mse_log_linear:.6f}")
print(f"Sigmoid: {mse_power:.6f}")

Logarithmic Forecast:
Step 90000: 52.7610
Step 100000: 53.6404
Step 110000: 54.4360
Step 120000: 55.1623

Log-Linear Forecast:
Step 90000: 49.9012
Step 100000: 49.6383
Step 110000: 49.2447
Step 120000: 48.7432

Power law Forecast:
Step 90000: 50.7325
Step 100000: 51.1155
Step 110000: 51.4451
Step 120000: 51.7326

Mean Squared Error:
Logarithmic: 3.938715
Power law: 1.191293
Sigmoid: 0.316578


In [19]:
colors = [
'#1f77b4',  # muted blue
'#ff7f0e',  # safety orange
'#2ca02c',  # cooked asparagus green
'#d62728',  # brick red
'#9467bd',  # muted purple
'#8c564b',  # chestnut brown
'#e377c2',  # raspberry yogurt pink
'#7f7f7f',  # middle gray
'#bcbd22',  # curry yellow-green
'#17becf'   # blue-teal
]

In [21]:
# Extract step and bbox/AP columns
steps = data["Step"].values
ap_scores = data["bbox/AP"].values

# Fit the function
popt, pcov = curve_fit(power_func, steps, ap_scores, p0=[50, 50, 0.1])

MAX = 375000
# Generate points for the fitted curve
x_fit = np.linspace(steps.min(), MAX + 2500, 1000)
y_fit = power_func(x_fit, *popt)

# Calculate confidence intervals
alpha = 0.05  # 95% confidence interval
n = len(steps)
p = len(popt)  # number of parameters
dof = max(0, n - p)  # degrees of freedom

# Student's t-distribution for alpha/2 and dof
tval = np.abs(stats.t.ppf(alpha / 2, dof))

# Calculate standard errors
sigma = np.sqrt(np.diag(pcov))

# Calculate confidence intervals
ci = tval * sigma

# Generate upper and lower confidence bounds
lower = power_func(x_fit, *(popt - ci))
upper = power_func(x_fit, *(popt + ci))

# Create the plot
fig = go.Figure()

# Add the actual data points
fig.add_trace(go.Scatter(x=steps, y=ap_scores, mode="markers", name="Actual Data"))

# Add the fitted curve
fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode="lines", name="Power Law Fit"))

# Add the confidence intervals
fig.add_trace(
    go.Scatter(
        x=np.concatenate([x_fit, x_fit[::-1]]),
        y=np.concatenate([upper, lower[::-1]]),
        fill="toself",
        fillcolor="rgba(0,100,80,0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip",
        showlegend=False,
    )
)

# Update layout
fig.update_layout(
    title="Object Detection Model Performance with Power Law Fit",
    xaxis_title="Steps",
    yaxis_title="bbox/AP",
    legend_title="Legend",
    hovermode="x",
)
# plot one point corresponding to MAX
y_MAX = power_func(MAX, *popt)
fig.add_trace(
    go.Scatter(
        x=[MAX],
        y=[y_MAX],
        mode="markers",
        name=f"50ep {y_MAX:.2f}",
        marker=dict(size=10, color="red"),
    )
)

# Show the plot
fig.show()

# Print the model parameters and their confidence intervals
print("Model Parameters:")
param_names = ["a", "b", "c"]
for name, value, error in zip(param_names, popt, ci):
    print(f"{name}: {value:.4f} ± {error:.4f}")

# Forecast for future steps
future_steps = np.array([90000, MAX])
forecasts = power_func(future_steps, *popt)

print("\nForecasts:")
for step, forecast in zip(future_steps, forecasts):
    print(f"Step {step}: {forecast:.4f}")

Model Parameters:
a: 58.2154 ± 1.7620
b: 2212.7015 ± 835.9942
c: 0.4987 ± 0.0505

Forecasts:
Step 90000: 50.7325
Step 375000: 54.5429
